# LIBRARIES

In [1]:
library(tidyverse)
library(repr)
library(broom)
library(leaps)
library(moderndive)
library(MASS)
library(car)
library(rsample)
print("LIBRARIES LOADED")

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘MASS’


The following object is masked from ‘package:dplyr’:

    select


Loading required package: carData


Attaching package: ‘car’


The following object is masked from ‘package:dplyr’:

    recode


The following object is masked from ‘package:purrr’:

    some




[1] "LIBRARIES LOADED"


# DATA AND TIDYING

In [17]:
file_url <- "https://drive.google.com/uc?export=download&id=1ZjZvLl5dUzHEF8ouimlTg8t0MorhjzVA"
sleep_data <- read.csv(file_url)
head(sleep_data)
set.seed(114514) # SEED, DO NOT CHANGE

,Person.ID,Gender,Age,Occupation,Sleep.Duration,Quality.of.Sleep,Physical.Activity.Level,Stress.Level,BMI.Category,Blood.Pressure,Heart.Rate,Daily.Steps,Sleep.Disorder
,<int>,<chr>,<int>,<chr>,<dbl>,<int>,<int>,<int>,<chr>,<chr>,<int>,<int>,<chr>
1,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,None
2,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,None
3,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,None
4,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
5,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
6,6,Male,28,Software Engineer,5.9,4,30,8,Obese,140/90,85,3000,Insomnia


In [20]:
sleep_data_reduced <- sleep_data %>%
  dplyr::select(-any_of(c("Person.ID", "Gender", "Blood.Pressure", "Heart.Rate", "Daily.Steps")))
new_names <- c(
  "Age", "Occupation", "Sleep_Duration", 
  "Quality_of_Sleep", "Physical_Activity_Level", "Stress_Level", 
  "BMI_Category",
  "Sleep_Disorder"
)
names(sleep_data_reduced)<- new_names
sleep_data_reduced<- sleep_data_reduced|>mutate(BMI_Category = as.factor(BMI_Category), Occupation = as.factor(Occupation))
sleep_data_reduced<- sleep_data_reduced|>mutate(Sleep_Disorder= ifelse(Sleep_Disorder=="None", "False", "True"))
sleep_data_reduced<- sleep_data_reduced|>mutate(Sleep_Disorder = as.factor(Sleep_Disorder))
head(sleep_data_reduced) 
nrow(sleep_data_reduced)

,Age,Occupation,Sleep_Duration,Quality_of_Sleep,Physical_Activity_Level,Stress_Level,BMI_Category,Sleep_Disorder
,<int>,<fct>,<dbl>,<int>,<int>,<int>,<fct>,<fct>
1,27,Software Engineer,6.1,6,42,6,Overweight,False
2,28,Doctor,6.2,6,60,8,Normal,False
3,28,Doctor,6.2,6,60,8,Normal,False
4,28,Sales Representative,5.9,4,30,8,Obese,True
5,28,Sales Representative,5.9,4,30,8,Obese,True
6,28,Software Engineer,5.9,4,30,8,Obese,True


[1] 374

# IMPLEMENTATION
QUESTION: ASSOCIATION BETWEEN SLEEP DURATION (RESPONSE) AND OTHER VARIABLES.

HERE WE SPLIT THE DATA TO TRAINING/TESTING, 70/30 BEFORE DOING THE FIRST VIF CHECK.

In [33]:
# SPLITTING
# SPLIT DATA BEFORE FIRST VIF CHECK

data_split <- sleep_data_reduced |> initial_split(prop = 0.7, strata = Sleep_Duration)
sleep_train <- training(data_split)
sleep_test <- testing(data_split)
print(paste("TRAINING N-ROWS", nrow(sleep_train)))
print(paste("TESTING N-ROWS", nrow(sleep_test)))

[1] "TRAINING N-ROWS 260"
[1] "TESTING N-ROWS 114"


In [28]:
# FIRST VIF CHECK
sleep_full <- lm(Sleep_Duration~., data=sleep_train)
vif(sleep_full)

,GVIF,Df,GVIF^(1/(2*Df))
Age,7.801410,1,2.793100
Occupation,43.727605,9,1.233540
Quality_of_Sleep,24.223310,1,4.921718
Physical_Activity_Level,1.732772,1,1.316348
Stress_Level,14.733625,1,3.838440
BMI_Category,14.049277,3,1.553373
Sleep_Disorder,3.914628,1,1.978542


High scaled-GVIF:

Quality_of_Sleep, Stress_Level

In [30]:
# STEP AIC
aic_model <- stepAIC(sleep_full, direction = "backward", k = log(nrow(sleep_train)))
summary(aic_model)

Start:  AIC=-560.67
Sleep_Duration ~ Age + Occupation + Quality_of_Sleep + Physical_Activity_Level + 
    Stress_Level + BMI_Category + Sleep_Disorder

                          Df Sum of Sq    RSS     AIC
- BMI_Category             3    0.1956 20.672 -574.88
- Age                      1    0.0031 20.480 -566.19
- Sleep_Disorder           1    0.1354 20.612 -564.52
<none>                                 20.476 -560.67
- Quality_of_Sleep         1    0.7659 21.242 -556.69
- Physical_Activity_Level  1    2.7586 23.235 -533.37
- Stress_Level             1    3.4964 23.973 -525.25
- Occupation               9   11.9036 32.380 -491.57

Step:  AIC=-574.88
Sleep_Duration ~ Age + Occupation + Quality_of_Sleep + Physical_Activity_Level + 
    Stress_Level + Sleep_Disorder

                          Df Sum of Sq    RSS     AIC
- Age                      1    0.0291 20.701 -580.08
- Sleep_Disorder           1    0.0605 20.733 -579.68
<none>                                 20.672 -574.88
- Quality


Call:
lm(formula = Sleep_Duration ~ Occupation + Quality_of_Sleep + 
    Physical_Activity_Level + Stress_Level, data = sleep_train)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.72599 -0.18718 -0.06338  0.19325  1.00551 

Coefficients:
                                Estimate Std. Error t value Pr(>|t|)    
(Intercept)                     6.153200   0.579325  10.621  < 2e-16 ***
OccupationDoctor                0.724477   0.074562   9.716  < 2e-16 ***
OccupationEngineer              0.661247   0.073493   8.997  < 2e-16 ***
OccupationLawyer                0.377713   0.080239   4.707 4.18e-06 ***
OccupationNurse                 0.184055   0.075662   2.433 0.015701 *  
OccupationSales Representative  0.716432   0.313788   2.283 0.023270 *  
OccupationSalesperson           0.430080   0.093035   4.623 6.11e-06 ***
OccupationScientist             0.383741   0.197964   1.938 0.053709 .  
OccupationSoftware Engineer     0.397608   0.158845   2.503 0.012957 *  
OccupationTeacher

In [31]:
# 2ND VIF WITH AIC MODEL
vif(aic_model)

,GVIF,Df,GVIF^(1/(2*Df))
Occupation,5.195102,9,1.095860
Quality_of_Sleep,13.791710,1,3.713719
Physical_Activity_Level,1.548737,1,1.244482
Stress_Level,11.900038,1,3.449643


REMOVED AFTER STEP-AIC:

Age, BMI_Category, Sleep_Disorder.

# VALIDATION

In [36]:
pred_test <- predict(aic_model, newdata = sleep_test)

actual <- sleep_test$Sleep_Duration
resid <- actual - pred_test

RMSE <- sqrt(mean(resid^2))
MAE <- mean(abs(resid))
R2 <- 1 - sum(resid^2) / sum((actual - mean(actual))^2)

print(c(RMSE = RMSE, MAE = MAE, R2 = R2))

mean_sleep <- mean(sleep_test$Sleep_Duration)
relative_RMSE <- RMSE / mean_sleep
relative_MAE <- MAE / mean_sleep

print(paste("RMSE RELATIVE TO MEAN SLEEP DURATION: ", relative_RMSE, ", good."))
print(paste("MAE RELATIVE TO MEAN SLEEP DURATION: ", relative 

     RMSE       MAE        R2 
0.2839613 0.2147667 0.8723937 


Summary of test-set validation:

RMS error: about 0.284 hours sleep duration or 17 minutes.
MA error: 0.215 hours or 13 minutes.
R^2: 0.872, good.